# OilyGiant Project 

## Introduction

This project aims to determine the best locations to open 200 new oil wells in three candidate regions, maximizing profit margins and minimizing associated risks. To achieve this, geological data from each region will be analyzed using a linear regression model, which will allow for the prediction of reserve volumes in thousands of barrels for new wells. <br><br>

The analysis will include the selection of wells with the highest production potential and a detailed assessment of benefits and risks using statistical techniques such as bootstrapping. Based on the results, the most promising region will be proposed for development, ensuring a loss risk of less than 2.5% and optimizing economic returns within the established budget of $100 million. <br><br>

This structured, data-driven approach ensures informed decision-making for the sustainable development of energy resources.

## Development

### Inspect data

In [ ]:
# Library import 

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error


In [28]:
# Import data

df_0= pd.read_csv('/Users/pauli/Documents/Data/oily_giant/geo_data_0.csv')
df_1= pd.read_csv('/Users/pauli/Documents/Data/oily_giant/geo_data_1.csv')
df_2= pd.read_csv('/Users/pauli/Documents/Data/oily_giant/geo_data_2.csv')

In [29]:
# Inspect datasets

df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [30]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [31]:
df_0.duplicated().sum()

0

There is no duplicate data or null data, the data types correspond in DF 0.

In [32]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [33]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [34]:
df_1.duplicated().sum()

0

There is no duplicate data or null data, the data types correspond in DF 1.

In [35]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [36]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [37]:
df_2.duplicated().sum()

0

In [38]:
# ID values ​​are removed from all DFs as they may be counterproductive to the model and do not identify an important feature.

df_0 = df_0.drop(['id'], axis=1)
df_1= df_1.drop(['id'], axis=1)
df_2= df_2.drop(['id'], axis=1)

In [39]:
#Check that it is correct

print (df_0.head(2))
print (df_1.head(2))
print (df_2.head(2))

         f0        f1       f2     product
0  0.705745 -0.497823  1.22117  105.280062
1  1.334711 -0.340164  4.36508   73.037750
          f0        f1        f2    product
0 -15.001348 -8.276000 -0.005876   3.179103
1  14.272088 -3.475083  0.999183  26.953261
         f0        f1        f2    product
0 -1.146987  0.963328 -0.828965  27.758673
1  0.262778  0.269839 -2.530187  56.069697


### Model Training

The model is trained with functions, so that it is easy to apply to all DFs.

In [40]:
#Prepare data for training

def prepare_data(df):

    features = df.drop(['product'], axis=1)
    target = df['product']
    return features, target


In [45]:
#Divide the data into training and validation, train the model.

def train_and_validate(features, target):
    """
    Divides the data into training and validation (75:25).
    Returns the trained model, predictions, actual responses, and RMSE.
    """
    X_train, X_valid, y_train, y_valid = train_test_split(features, target, test_size=0.25, random_state=15)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    predictions = model.predict(X_valid)
    rmse = root_mean_squared_error(y_valid, predictions)
    
    return model, predictions, y_valid, rmse

In [46]:
#Results analysis

def analyze_results(predictions, y_valid):
    """
Analyze the model results, showing the predicted average booking volume and RMSE.
"""
    avg_prediction = predictions.mean()
    avg_actual = y_valid.mean()
    print(f"Predicted average booking volume: {avg_prediction:.2f}")
    print(f"Average real booking volume: {avg_actual:.2f}")

In [47]:
# Implementar el proceso 

def process_region(data):
    """
    Complete the entire process for a specific region:
Prepare the data.
Train and validate the model.
Analyze the results.
    """
    features, target = prepare_data(data)
    model, predictions, y_valid, rmse = train_and_validate(features, target)
    analyze_results(predictions, y_valid)
    print(f"RMSE: {rmse:.2f}")
    return model, predictions, y_valid



In [48]:
# Procesar cada región

results_0 = process_region(df_0)
results_1 =process_region(df_1)
results_2 =process_region(df_2)

Predicted average booking volume: 92.64
Average real booking volume: 92.42
RMSE: 37.69
Predicted average booking volume: 68.48
Average real booking volume: 68.48
RMSE: 0.89
Predicted average booking volume: 95.24
Average real booking volume: 95.29
RMSE: 40.19


In Region 0: The predicted mean reserves volume (92.42) is very close to the actual value, however, the RMSE (37.69) is relatively high, suggesting significant variability in the predictions. <br>
In Region 1: The predicted mean reserves volume exactly matches the actual value, and the RMSE is very low (0.89), indicating that the predictions are consistently accurate and have little dispersion. <br>
In Region 2: The RMSE (40.19) is the highest of the three regions, suggesting that the model struggles to accurately predict at the individual level in this region.

### Calculation of profits.

In [49]:
# Conditions

budget = 100000000 #Investment budget
num_wells = 200 #Number of wells
min_profit_threshold = 111.1 #Minimum amount of barrel units to avoid losses
unit_profit = 4500 #Income per barrel in USD



def calculate_profit(predictions, num_wells, unit_profit, budget):
    """
    Calculate the estimated net profit by selecting the most promising wells."""
    
       # Select the wells with the largest estimated reserves
    top_predictions = np.sort(predictions)[-num_wells:]
    
    # Calculate the total reserve volume of the selected wells
    total_reserves = top_predictions.sum()
    
    # Calculate total income
    revenue = total_reserves * unit_profit
    
    # Calculate the net profit
    profit = revenue - budget
    
    return profit



In [50]:
#Function application

profit_df_0= calculate_profit(results_0[1], num_wells, unit_profit, budget)
profit_df_1= calculate_profit(results_1[1], num_wells, unit_profit, budget)
profit_df_2= calculate_profit(results_2[1], num_wells, unit_profit, budget)

print(f"Ganancia Region 0: ${profit_df_0:,.2f}")
print(f"Ganancia Region 1: ${profit_df_1:,.2f}")
print(f"Ganancia Region 2: ${profit_df_2:,.2f}")

Ganancia Region 0: $39,327,676.73
Ganancia Region 1: $24,816,738.80
Ganancia Region 2: $33,239,809.68


To this point, all three regions are profitable to varying degrees. Region zero has the highest profit among the three analyzed, which could be due to a higher average reserve volume or better predictions in the selected wells.

### Risks and rewards for each region.

In [51]:
def bootstrap_profit(predictions, num_wells, unit_profit, budget, n_iterations=1000):
    """
    Applies the bootstrapping technique to calculate the distribution of benefits.
    """
    profits = []
    for _ in range(n_iterations):
        # Generate a random sample with replacement of the predictions
        sample = np.random.choice(predictions, size=len(predictions), replace=True)
        
        # Calculate the profit with the generated sample
        profit = calculate_profit(sample, num_wells, unit_profit, budget)
        profits.append(profit)
    
    # Convert the list of benefits to a numpy array to make calculations easier
    profits = np.array(profits)
    
    # Average profit
    avg_profit = profits.mean()
    
    # 95% confidence interval
    confidence_interval = np.percentile(profits, [2.5, 97.5])
    
    # Risk of loss (percentage of simulations with negative profit)
    loss_risk = (profits < 0).mean() * 100
    
    return {
        "average_profit": avg_profit,
        "confidence_interval": confidence_interval,
        "loss_risk": loss_risk
    }

# Applying bootstrapping to each region
bootstrap_results_0 = bootstrap_profit(results_0[1], num_wells, unit_profit, budget)
bootstrap_results_1 = bootstrap_profit(results_1[1], num_wells, unit_profit, budget)
bootstrap_results_2 = bootstrap_profit(results_2[1], num_wells, unit_profit, budget)

# Show results
print("\nBootstrapping Results by Region:")
for i, results in enumerate([bootstrap_results_0, bootstrap_results_1, bootstrap_results_2]):
    print(f"\nRegión {i}:")
    print(f"Average profit: ${results['average_profit']:,.2f}")
    print(f"95% confidence interval: ${results['confidence_interval'][0]:,.2f} - ${results['confidence_interval'][1]:,.2f}")
    print(f"Loss risk: {results['loss_risk']:,.2f}%")


Bootstrapping Results by Region:

Región 0:
Average profit: $39,319,931.58
95% confidence interval: $38,150,912.33 - $40,529,896.83
Loss risk: 0.00%

Región 1:
Average profit: $24,814,662.14
95% confidence interval: $24,762,749.67 - $24,871,326.43
Loss risk: 0.00%

Región 2:
Average profit: $33,221,063.85
95% confidence interval: $32,316,278.17 - $34,151,158.86
Loss risk: 0.00%


## Conclusions.

According to the results, Region 0 has the highest average profit and a narrow confidence interval, indicating high reliability in the estimated profits. On the other hand, Region 1 has the lowest average profit among the three regions, but the confidence interval is extremely narrow, suggesting highly predictable behavior in this region. Finally, Region 2 has a considerably high average profit, although lower than that of Region 0. Its confidence interval is also reasonably narrow, indicating reliability. <br> <br>
In conclusion, investing in Region 0 is recommended. This approach would maximize expected average profits while maintaining zero risk of loss and a high level of confidence. The high profitability in this region is consistent and supported by a narrow confidence interval.

